### The **Tricks** I should have known earlier :(

### Some hands on libs

#### [logging](https://docs.python.org/3/howto/logging.html)

```
import logging

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logging.info(f'''Network INFO:
        Latent variable:   {Z_CHS} channels
        Hidden variable:   {H_CHS} channels {H_SIZE} size
        Embedding of Lv:   {PHIZ_CHS} channels
        Embedding of It:   {PHIX_CHS} channels
        ''')
```

#### [tqdm](https://tqdm.github.io/)

```
from tqdm import tqdm

with tqdm(total:int, unit:'str') as pbar:
    for:
        ...
        pbar.update(1)
```

### The **Questions** I have asked 1k+ times on StackOverflow ;)

- Saving images in Python at a very high quality in `matplotlib`, [here](https://stackoverflow.com/questions/16183462/saving-images-in-python-at-a-very-high-quality).

- Fast method to retrieve contour mask from a binary mask in Python using `cv2`, [here](https://stackoverflow.com/questions/40441910/fast-method-to-retrieve-contour-mask-from-a-binary-mask-in-python).

- `pickle` error: `OverflowError: cannot serialize a bytes object larger than 4 GiB`, run with [`protocol = 4`](https://stackoverflow.com/questions/29704139/pickle-in-python3-doesnt-work-for-large-data-saving)

### Medical Image Registration Methods (conventional)

- Demons and its variations, [link](https://simpleitk.readthedocs.io/en/master/link_DemonsRegistration2_docs.html#lbl-demons-registration2).

- SyN in ANTsPy implementation, [link](https://antspy.readthedocs.io/en/latest/registration.html).

### SyN Registration Template, param details in [link](https://antspy.readthedocs.io/en/latest/registration.html).

In [ ]:
import ants
import nibabel as nib
from torch.utils import data
import numpy as np 

from data import Sample_Dataset 

dataset = Sample_Dataset()
dataloader = data.DataLoader(dataset=dataset, batch_size=1, shuffle=False)

for ind, items in enumerate(dataset):
    # *Get data -> numpy.array shape [C, H, W]
    moving = items[0].numpy().astype('float32')
    fixed = items[1].numpy().astype('float32')
    # * numpy.array -> ants format 
    moving_img = ants.from_numpy(moving)
    fixed_img = ants.from_numpy(fixed)
    # *SyN registration part 
    out = ants.registration(
        fixed=fixed_img, 
        moving=moving_img, 
        type_of_transform='SyN',
        reg_iterations=(100, 100, 100), #* vector of iterations for syn.
        flow_sigma=5, 
        grad_step=0.8
        )
    # *Get moved image and transform to numpy.array
    # *shape [C, H, W] 
    mvout_np = out['warpedmovout'].numpy()
    # *Get forward path (moving -> fixed) transformation dir. 
    fwd_path = out['fwdtransforms'][0]
    # *Load transformation matrix (stored as .nii) file.
    try:
        transform = nib.load(fwd_path)
    except:
        print(f'empty tranform at {items[-1][0]}')
        continue
    transform_np = transform.get_fdata().squeeze()
    transform_np = np.transpose(transform_np,(2,0,1)) #* shape [C, H, W]
    # *Get Jacobian Determinant matrix 
    jac = ants.create_jacobian_determinant_image(
        fixed_img,
        out['fwdtransforms'][0],
        do_log = False
        )
    # *numpy form 
    jac_np = jac.numpy()

### Spatial Transformer Network (STN) - differentialble sampler 
- Template for own usage, original implementation, check out [link](https://github.com/Kh4n/SpatialTransformer).

- Usage of grid sampler, check out [link](https://pytorch.org/docs/stable/generated/torch.nn.functional.grid_sample.html).

In [2]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

class SpatialTransformer(nn.Module):
    """
    N-D Spatial Transformer
    """

    def __init__(self, size, mode='bilinear'):
        super().__init__()

        self.mode = mode

        # create sampling grid
        vectors = [torch.arange(0, s) for s in size]
        grids = torch.meshgrid(vectors)
        grid = torch.stack(grids)

        grid = torch.unsqueeze(grid, 0)
        grid = grid.type(torch.FloatTensor)

        # registering the grid as a buffer cleanly moves it to the GPU, but it also
        # adds it to the state dict. this is annoying since everything in the state dict
        # is included when saving weights to disk, so the model files are way bigger
        # than they need to be. so far, there does not appear to be an elegant solution.
        # see: https://discuss.pytorch.org/t/how-to-register-buffer-without-polluting-state-dict
        self.register_buffer('grid', grid)

    def forward(self, src, flow):
        '''
        flow -> [B, C(2), U(X-COL), V(Y-ROW)]
        src -> [B, C(1/3), H, W]
        '''
        # *new locations
        new_locs = self.grid + flow
        shape = flow.shape[2:] 

        # *need to normalize grid values to [-1, 1] for resampler
        for i in range(len(shape)):
            new_locs[:, i, ...] = 2 * (new_locs[:, i, ...] / (shape[i] - 1) - 0.5)

        # *move channels dim to last position
        # * grid sampler take src input shape N C H W and grid shape N H W 2 for 4D input case. 
        # B, H, W, C[U(X), V(Y)]
        new_locs = new_locs.permute(0, 2, 3, 1) 
        #C[V(Y-ROW-H), U(X-COL-W)]
        new_locs = new_locs[..., [1, 0]] 
        
        return F.grid_sample(src, new_locs, align_corners=True, mode=self.mode)

if __name__ == '__main__':
    stn = SpatialTransformer(size=(128,128),mode='bilinear')
    src_test = torch.randn(1,1,128,128)
    def_test = torch.randn(1,2,128,128)
    moved_src = stn(src_test, def_test)
    print(moved_src.shape)

torch.Size([1, 1, 128, 128])
